# Controlling Keithley with Prologix

## Talking to prologix

What I want to do with `PrologixResourceManager`:

```python
p = PrologixResourceManager('lightwave-lab-prologix1.princeton.edu')

p.connect()  # connects to socket and leaves it open
p.startup()  # configures prologix to communicate via gpib
p.send('++addr 23')  # talks to address 23
p.send('command value')  # sends the command and does not expect to read anything
p.query('command')  # sends a command but reads stuff back
p.disconnect()
```

The problem with the above is that if there is any error with `startup`, `send` or `query`, the `disconnect` method will not be called. So let's code a decorator as such:

```python
@contextmanager
def connected(self):
    self.connect()
    try:
        yield self
    finally:
        self.disconnect()
```

Then we can do the previous code like this:

```python
p = PrologixResourceManager('lightwave-lab-prologix1.princeton.edu')

with p.connected():
    p.startup()
    p.send('++addr 23')  # talks to address 23
    p.send('command value')  # sends the command and does not expect to read anything
    p.query('command')  # sends a command but reads stuff back
```

If we try to send a message without the decorator, then we should connect and disconnect right before.
```python
p = PrologixResourceManager('lightwave-lab-prologix1.princeton.edu')

p.send('++addr 23')  # opens and close socket automatically
```

In [1]:
%load_ext autoreload
%autoreload 2
from prologix_temp import PrologixResourceManager

In [2]:
p = PrologixResourceManager('lightwave-lab-prologix1.ee.princeton.edu')

In [3]:
print('querying version')
print(repr(p.query('++ver')))
print(p._socket)  # socket should be None

querying version
'Prologix GPIB-ETHERNET Controller version 01.06.06.00\r\n'
None


In [4]:
# querying version inside with statement
with p.connected():
    print(repr(p.query('++ver')))  # should print 'Prologix.... \r\n'
    print(p._socket)  # socket should not be None
    print(repr(p.query('++addr')))  # should print '23\r\n'
print(p._socket)  # socket should be None

'Prologix GPIB-ETHERNET Controller version 01.06.06.00\r\n'
<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 40510), raddr=('128.112.48.58', 1234)>
'23\r\n'
None


In [5]:
# nested with statement.
with p.connected():
    print(repr(p.query('++ver')))  # should print 'Prologix.... \r\n'
    print(p._socket)  # socket should not be None
    with p.connected():
        print(repr(p.query('++ver')))  # should print 'Prologix.... \r\n'
        print(p._socket)  # socket should not be None
    print(p._socket)  # socket should not be None
print(p._socket)  # socket should be None

'Prologix GPIB-ETHERNET Controller version 01.06.06.00\r\n'
<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 40512), raddr=('128.112.48.58', 1234)>
'Prologix GPIB-ETHERNET Controller version 01.06.06.00\r\n'
<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 40512), raddr=('128.112.48.58', 1234)>
<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 40512), raddr=('128.112.48.58', 1234)>
None


In [6]:
print(p.send('*IDN?'))
print(p.query('++read eoi'))

None
KEITHLEY INSTRUMENTS INC.,MODEL 2400,4050948,C32   Oct  4 2010 14:20:11/A02  /U/K



In [8]:
# Testing errors
from socket import gaierror

perr = PrologixResourceManager('lightwavexxxxx-lab-prologix1.ee.princeton.edu')
try:
    with perr.connected():  # should throw an error right here
        pass
except gaierror:
    print("successfully detected gaierror")

successfully detected gaierror


## Visa object

First, it would be nice to know whether a hostname is valid or not. That's why we are going to implement first a `_validate_hostname`. A hostname can be given as an ip address or a dns name. It would be nice to validate that a priori without attempting any connection, just as a sanity check.

In [9]:
from prologix_temp import _validate_hostname

In [10]:
trials = ['princeton.edu', 
          'princeton.edu.123', 
          'a.b.c', 
          'a.b',
          'abc', 
          '12.132.132.1', 
          '0.0.0.0',
          '0.0.0.0.0',
          '127.0.0.1',
          '300.0.0.1',
         ]
for trial in trials:
    print(_validate_hostname(trial), '\t', trial)

True 	 princeton.edu
False 	 princeton.edu.123
True 	 a.b.c
True 	 a.b
True 	 abc
True 	 12.132.132.1
True 	 0.0.0.0
False 	 0.0.0.0.0
True 	 127.0.0.1
False 	 300.0.0.1


In [11]:
from prologix_temp import PrologixGPIBObject

In [12]:
keithley_test = \
    PrologixGPIBObject('prologix://lightwave-lab-prologix1.princeton.edu/23', 
                       tempSess=True)

In [13]:
keithley_test.instrID()

'KEITHLEY INSTRUMENTS INC.,MODEL 2400,4050948,C32   Oct  4 2010 14:20:11/A02  /U/K'

In [14]:
keithley_test.query('OUTP:STATE?')

'0'

## TODO before we can include in lightlab

1. PrologixGPIBObject must have the following signature (same as VISAObject).

```python
class PrologixGPIBObject:
    def __init__(self, address=None, tempSess=False):
    def open(self):
    def close(self):
    def write(self, writeStr):
    def query(self, queryStr, withTimeout=None):
    def instrID(self):
    @property
    def timeout(self):
    @timeout.setter
    def timeout(self, newTimeout):
    def reset(self):
    def wait(self, bigMsTimeout=10000):
```

2. Additional methods for both VISAObject and PrologixGPIBObject

```python
def _spoll(self):
    '''Return SRQ status byte of the instrument.'''

def _LLO(self):
    '''This command disables front panel operation of the currently addressed instrument.'''

def _LOC(self):
    '''This command enables front panel operation of the currently addressed instrument.'''

@property
def termination(self):
    '''Termination GPIB character. Options: '\r\n', '\r', '\n', ''. '''
    
def clear(self):
    '''This command sends the Selected Device Clear (SDC) message to the currently specified GPIB address.'''
    
def query_raw_binary(self, queryStr, withTimeout):
    '''Read the unmodified string sent from the instrument to the computer. In contrast to read(), no termination characters are stripped. Also no decoding.'''
```

## Also need to test for concurrency.

What happens if two connections to the prologix are attempted at the same time?

``` python
p1 = PrologixResourceManager('lightwave-lab-prologix1.ee.princeton.edu')
p2 = PrologixResourceManager('lightwave-lab-prologix1.ee.princeton.edu')

with p1.connected(), p2.connected():
    print(p1.query('++ver'))  # this must work
    print(p2.query('++ver'))  # this must work
```

Study what happens if you start socket, unplug ethernet cable, and write a query. Socket must be closed at the end.

Study what happens if you start socket, unplug and replug ethernet cable, and write a query.

Study what happens if you start socket, then connect via nc tool outside, then write a query. Which sockets get destroyed? What errors are generated? Write here so we include in the documentation.

- No errors are generated, but all queries will automatically return empty. This means that sockets should not stay open for very long.

Also two instrument instances must be independent.

``` python
i1 = PrologixGPIBObject('address1')
i2 = PrologixGPIBObject('address2')

print(i1.instrID())  # shows id of instrument 1
print(i2.instrID())  # shows id of instrument 2
print(i1.instrID())  # shows id of instrument 1

```

In [28]:
p1 = PrologixResourceManager('lightwave-lab-prologix1.ee.princeton.edu')
p2 = PrologixResourceManager('lightwave-lab-prologix1.ee.princeton.edu')

with p1.connected(), p2.connected():
    print(p1._socket)
    print(p2._socket)
    print('1.' + p1.query('++ver'))  # this must work
    print('2.' + p2.query('++ver'))  # this must work
    print(p1._socket)
    print(p2._socket)

<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 53518), raddr=('128.112.48.58', 1234)>
<socket.socket fd=59, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 53520), raddr=('128.112.48.58', 1234)>
1.
2.Prologix GPIB-ETHERNET Controller version 01.06.06.00

<socket.socket fd=57, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 53518), raddr=('128.112.48.58', 1234)>
<socket.socket fd=59, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.112.48.53', 53520), raddr=('128.112.48.58', 1234)>
